# Import libraries

In [ ]:
import requests
import json
import pandas as pd
import base64
import ipywidgets as widgets
from ipywidgets import Layout
from PIL import Image
import numpy as np
import rioxarray as rxr
import xarray as xr
from urllib.request import urlopen
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import hvplot.pandas
import hvplot.xarray
import holoviews as hv
import holoviews.operation.datashader as hd
import bokeh as bk
from holoviews import opts
hv.extension('bokeh')

import warnings
warnings.filterwarnings('ignore')

In [ ]:
%load_ext autoreload
%autoreload 2
import WEkEO_methods as m

- Reference: https://www.wekeo.eu/docs/harmonised-data-access-api
- API: https://wekeo-broker.apps.mercator.dpi.wekeo.eu/databroker/ui/#/

## Get token

Reference used to get the access token follow this page instructions: 
- https://curlconverter.com/ to translate the curl request
- https://www.base64encode.org/ write "username:password" and copy&paste the base64encore in the headers below.

The following widget allows to login with your WEkEO credentials and obtain the token with the **get_token** function. Just insert Username and Password without launching the script again (also for other widgets just input the data without launching the code again):

In [ ]:
username = m.text_widget("Insert username: ")
password = m.password_widget("Insert password: ")

login_box = widgets.VBox([username, password])
login_box

In [ ]:
headers = m.get_token(username, password)
token = list(headers.values())[0]

List of datasets used for this notebook:

In [ ]:
ds_list = ['EO:ECMWF:DAT:REANALYSIS_ERA5_SINGLE_LEVELS',
          'EO:ECMWF:DAT:CAMS_EUROPE_AIR_QUALITY_FORECASTS',
          'EO:ECMWF:DAT:EFAS_HISTORICAL',
          'EO:MO:DAT:NWSHELF_ANALYSISFORECAST_PHY_LR_004_001:cmems_mod_nws_phy-bottomt_anfc_7km-2D_P1D-m']

----

# 1- ERA5 - Reanalysis Single Levels

Select the ERA5 Reanalysis Single Levels dataset from the dropdown:

In [ ]:
dataset_id = m.get_dropdown(ds_list, "List of datasets: ")
dataset_id

In [ ]:
data_df = m.get_data_from_name_request(2000, dataset_id.value)
dataset = data_df[data_df['datasetId'] == dataset_id.value]

In [ ]:
title = dataset.title.values[0]

In [ ]:
description = m.get_description(data_df, dataset_id)

In [ ]:
image = m.display_image(data_df, dataset_id, 500, "jpg", 600 )

In [ ]:
metadata = m.get_metadata(dataset_id, headers)

In [ ]:
tit = widgets.HTML(value='<b><font size="+2">'+title+'</font></b>')
descr = widgets.HTML('<details>'+description+'</details>')
image_box = widgets.VBox([image])
descr_box = widgets.VBox([tit, descr])


ui = widgets.AppLayout(header=descr_box,
          center = image_box,
          layout=widgets.Layout(border='solid'))

# compute_button.on_click(compute_button_f)
container = widgets.Box([ui])
display(container)

In [ ]:
variables_list = m.era5_single_levels_list(metadata)
format_type_list = list(metadata['parameters']['stringChoices'][0]['details']['valuesLabels'].keys())
product_type_list = list(metadata['parameters']['multiStringSelects'][1]['details']['groupedValueLabels'][0]['valuesLabels'].keys())
year_list = list(metadata['parameters']['multiStringSelects'][2]['details']['groupedValueLabels'][0]['valuesLabels'].keys())
month_list = list(metadata['parameters']['multiStringSelects'][3]['details']['groupedValueLabels'][0]['valuesLabels'].keys())
day_list = list(metadata['parameters']['multiStringSelects'][4]['details']['groupedValueLabels'][0]['valuesLabels'].keys())
time_list = list(metadata['parameters']['multiStringSelects'][5]['details']['groupedValueLabels'][0]['valuesLabels'].keys())
download_list = ["Download NETCDF", "Read NETCDF in memory"]

In [ ]:
params_sel = m.select_multiple(variables_list, "Variable: ")
product_type_sel = m.select_multiple(product_type_list, "Product type:")
year_sel = m.select_multiple(year_list, "Year: ")
month_sel = m.select_multiple(month_list, "Month: ")
day_sel = m.select_multiple(day_list, "Day: ")
time_sel = m.select_multiple(time_list, "Time: ")
format_type_sel = m.select_buttons(format_type_list, "Format: ", "netcdf")
download_sel = m.select_buttons(download_list, "Data download: ", "Read NETCDF in memory")

In [ ]:
title= widgets.HTML(value='<h2 style="text-align:center;">'+title+'</h2><hr>')
variables = widgets.VBox([title, params_sel, product_type_sel, year_sel, month_sel, day_sel, time_sel, format_type_sel,download_sel])
image_box = widgets.VBox([image])


ui = widgets.AppLayout(
          left_sidebar=variables,
          right_sidebar=image_box,
          layout=widgets.Layout(border='solid'))

# compute_button.on_click(compute_button_f)
container = widgets.Box([ui])
display(container)

In [ ]:
job = m.api_query_era5_single_levels(dataset_id, params_sel, year_sel, month_sel, day_sel, time_sel, product_type_sel, format_type_sel, token)

In [ ]:
jobId = job['jobId']
print("The job ID is: "+jobId)

In [ ]:
get_url = m.request_data(jobId, token)

Save files for netcdf only

In [ ]:
ds = m.download_type(download_sel, download_list, get_url)

In [ ]:
variables = list(ds.keys())
var_drop = m.get_dropdown(variables, "Variable: ")
var_drop

In [ ]:
dates = ds.time.values.astype('datetime64[s]').tolist()
date_drop = m.get_dropdown(dates, "Variable: ")
date_drop

In [ ]:
ds_data = ds[var_drop.value].sel(time=date_drop.value).squeeze()

In [ ]:
hv_dataset = hv.Dataset(ds_data)
hv_dataset.data

In [ ]:
map_height=400
map_width=800

hv_tiles_osm = hv.element.tiles.OSM()

hv.opts.defaults(
  hv.opts.Image(alpha=0.7,
                height=map_height, width=map_width, 
                colorbar=True, 
                tools=['hover'], active_tools=['wheel_zoom'],
                title=ds_data.long_name+" ["+ds_data.units+"]"),
  hv.opts.Tiles(active_tools=['wheel_zoom'], height=map_height, width=map_width)
)

hv_image_basic = hv.Image(hv_dataset)
hv_image_basic = hv_image_basic.options(cmap='inferno')
hv_image_basic

In [ ]:
ds = ds_data.rio.write_crs('EPSG:4326')
ds_mercator = ds.rio.reproject('EPSG:3857')

hv_dataset = hv.Dataset(ds_mercator)
hv_image_basic = hv.Image(hv_dataset)
hv_image_basic = hv_image_basic.options(cmap='inferno')

hv.element.tiles.OSM() * hv_image_basic

In [ ]:
ds.hvplot.quadmesh(
    'longitude', 'latitude', var_drop.value, projection=ccrs.Orthographic(-90, 30),
    global_extent=True, frame_height=540, cmap='viridis',
    coastline=True
)

In [ ]:
ds.hvplot.quadmesh(
    'longitude', 'latitude', var_drop.value, crs=ccrs.PlateCarree(), projection=ccrs.PlateCarree(),
    global_extent=True, frame_height=400, cmap='viridis',
    coastline=True
)

----

# 2 - CAMS - Europe Air Quality Forecasts

In [ ]:
dataset_id = m.get_dropdown(ds_list, "List of datasets: ")
dataset_id

Select 'EO:ECMWF:DAT:CAMS_EUROPE_AIR_QUALITY_FORECASTS' from the Dropdown list:

In [ ]:
data_df = m.get_data_from_name_request(2000, dataset_id.value)
dataset = data_df[data_df['datasetId'] == dataset_id.value]

In [ ]:
title = dataset.title.values[0]

In [ ]:
description = m.get_description(data_df, dataset_id)

In [ ]:
image = m.display_image(data_df, dataset_id, 500, "jpg", 600 )

In [ ]:
metadata = m.get_metadata(dataset_id, headers)

In [ ]:
tit = widgets.HTML(value='<b><font size="+2">'+title+'</font></b>')
descr = widgets.HTML('<details>'+description+'</details>')
image_box = widgets.VBox([image])
descr_box = widgets.VBox([tit, descr])


ui = widgets.AppLayout(header=descr_box,
          center = image_box,
          layout=widgets.Layout(border='solid'))

# compute_button.on_click(compute_button_f)
container = widgets.Box([ui])
display(container)

In [ ]:
format_type_list = list(metadata['parameters']['stringChoices'][0]['details']['valuesLabels'].keys())
params_list = list(metadata['parameters']['multiStringSelects'][0]['details']['groupedValueLabels'][0]['valuesLabels'].keys())
product_type_list = list(metadata['parameters']['multiStringSelects'][1]['details']['groupedValueLabels'][0]['valuesLabels'].keys())
level_list = list(metadata['parameters']['multiStringSelects'][2]['details']['groupedValueLabels'][0]['valuesLabels'].keys())
type_list = list(metadata['parameters']['multiStringSelects'][3]['details']['groupedValueLabels'][0]['valuesLabels'].keys())
hour_list = list(metadata['parameters']['multiStringSelects'][4]['details']['groupedValueLabels'][0]['valuesLabels'].keys())
leadtime_list = list(metadata['parameters']['multiStringSelects'][5]['details']['groupedValueLabels'][0]['valuesLabels'].keys())
download_list = ["Download NETCDF", "Read NETCDF in memory"]

In [ ]:
params_sel = m.select_multiple(params_list, "Variable:")
product_type_sel = m.select_multiple(product_type_list, "Product type:")
level_sel = m.select_multiple(level_list, "Level: ")
type_sel = m.select_multiple(type_list, "Type: ")
hour_sel = m.select_multiple(hour_list, "Hour: ")
leadtime_sel = m.select_multiple(leadtime_list, "Leadtime hour: ")
format_type_sel = m.select_buttons(format_type_list, "Format: ", "netcdf")
download_sel = m.select_buttons(download_list, "Data download: ", "Read NETCDF in memory")
start_date_sel = m.get_date_picker("Select start date: ")
end_date_sel = m.get_date_picker("Select start end: ")

In [ ]:
cams_map, dc = m.draw_map(45, 10, 4)

cams_map

In [ ]:
coords = dc.last_draw['geometry']['coordinates'][0]
W = coords[1][0]
E = coords[3][0]
N = coords[1][1]
S = coords[3][1]

In [ ]:
start_date = metadata['parameters']['dateRangeSelects'][0]['details']['start']
print("The start date is: "+start_date)

In [ ]:
title= widgets.HTML('<h2>CAMS European air quality forecasts </h2>')
variables = widgets.VBox([title, params_sel, product_type_sel, level_sel, type_sel, hour_sel, leadtime_sel, start_date_sel, end_date_sel, format_type_sel, download_sel])
image_box = widgets.VBox([image])


ui = widgets.AppLayout(
          left_sidebar=variables,
          right_sidebar=image_box,
          layout=widgets.Layout(border='solid'))

# compute_button.on_click(compute_button_f)
container = widgets.Box([ui])
display(container)

CHECK THE COORDINATES -> error in wekeo website as well.

In [ ]:
job = m.api_query_cams_forecast(dataset_id, params_sel, product_type_sel, level_sel, type_sel, hour_sel, leadtime_sel, start_date_sel, end_date_sel, format_type_sel, W, N, E, S, token)

In [ ]:
jobId = job['jobId']
print("The job ID is: "+jobId)

In [ ]:
get_url= m.request_data(jobId, token)

Save files for netcdf only

In [ ]:
ds = m.download_type(download_sel, download_list, get_url)
variables = list(ds.keys())
var_drop = m.get_dropdown(variables, "Variable: ")
var_drop

In [ ]:
ds_data = ds.squeeze()
ds_data

In [ ]:
hv_dataset = hv.Dataset(ds_data)

In [ ]:
ds.hvplot.quadmesh(
    'longitude', 'latitude', var_drop.value, projection=ccrs.Orthographic(4, 30),global_extent=True, frame_height=540, cmap='viridis',
    coastline=True
)

In [ ]:
ds.hvplot.quadmesh(
    'longitude', 'latitude', var_drop.value, crs=ccrs.PlateCarree(), projection=ccrs.PlateCarree(),
    global_extent=True, frame_height=400, cmap='viridis',
    coastline=True
)

----

# 3- EMERGENCY - EFAS Historical

In [ ]:
dataset_id = m.get_dropdown(ds_list, "List of datasets: ")
dataset_id

In [ ]:
data_df = m.get_data_from_name_request(2000, dataset_id.value)
dataset = data_df[data_df['datasetId'] == dataset_id.value]

In [ ]:
title = dataset.title.values[0]

In [ ]:
description = m.get_description(data_df, dataset_id)

In [ ]:
image = m.display_image(data_df, dataset_id, 500, "jpg", 600 )

In [ ]:
metadata = m.get_metadata(dataset_id, headers)

In [ ]:
tit = widgets.HTML(value='<b><font size="+2">'+title+'</font></b>')
descr = widgets.HTML('<details>'+description+'</details>')
image_box = widgets.VBox([image])
descr_box = widgets.VBox([tit, descr])


ui = widgets.AppLayout(header=descr_box,
          center = image_box,
          layout=widgets.Layout(border='solid'))

# compute_button.on_click(compute_button_f)
container = widgets.Box([ui])
display(container)

In [ ]:
metadata = m.get_metadata(dataset_id, headers)

In [ ]:
variable_list = list(metadata['parameters']['multiStringSelects'][0]['details']['groupedValueLabels'][0]['valuesLabels'].keys())
soil_level_list = list(metadata['parameters']['multiStringSelects'][1]['details']['groupedValueLabels'][0]['valuesLabels'].keys())
year_list = list(metadata['parameters']['multiStringSelects'][2]['details']['groupedValueLabels'][0]['valuesLabels'].keys())
month_list = list(metadata['parameters']['multiStringSelects'][3]['details']['groupedValueLabels'][0]['valuesLabels'].keys())
day_list = list(metadata['parameters']['multiStringSelects'][4]['details']['groupedValueLabels'][0]['valuesLabels'].keys())
time_list = list(metadata['parameters']['multiStringSelects'][5]['details']['groupedValueLabels'][0]['valuesLabels'].keys())
system_version_list = list(metadata['parameters']['stringChoices'][0]['details']['valuesLabels'].keys())
model_levels_list = list(metadata['parameters']['stringChoices'][1]['details']['valuesLabels'].keys())
formats_list = list(metadata['parameters']['stringChoices'][2]['details']['valuesLabels'].keys())
download_list = ["Download NETCDF"]

In [ ]:
formats = m.select_buttons(formats_list, "Choose format: ", formats_list[1]) 
system_version_sel = m.select_buttons(system_version_list, "System version:", system_version_list[-1])
model_levels_sel = m.select_buttons(model_levels_list, "Hydrological Model:", model_levels_list[-1])
variable_sel = m.select_multiple(variable_list, "Variable:")
year_sel = m.select_multiple(year_list, "Year:")
month_sel = m.select_multiple(month_list, "Month:")
day_sel = m.select_multiple(day_list, "Day:")
time_sel = m.select_multiple(time_list, "Time:")
soil_level_sel = m.select_multiple(soil_level_list, "Soil level:")
download_sel = m.select_buttons(download_list, "Data download: ", "Download NETCDF")

In [ ]:
title= widgets.HTML(value='<h2 style="text-align:center;">'+title+'</h2><hr>')
variables = widgets.VBox([title, system_version_sel, variable_sel, model_levels_sel, year_sel, month_sel, day_sel, time_sel, soil_level_sel, formats, download_sel])
image_box = widgets.VBox([image])

ui = widgets.AppLayout(
          left_sidebar=variables,
          right_sidebar=image_box,
          layout=widgets.Layout(border='solid'))

# compute_button.on_click(compute_button_f)
container = widgets.Box([ui])
display(container)

In [ ]:
# 02 may 2021 00:00 6hour discharge. works

In [ ]:
job = m.api_query_efas_historical(dataset_id, variable_sel, model_levels_sel, system_version_sel, year_sel, month_sel, day_sel, time_sel, soil_level_sel, formats, token)

In [ ]:
jobId = job['jobId']
print("The job ID is: "+jobId)

In [ ]:
get_url= m.request_data(jobId, token)

In [ ]:
ds = m.download_type(download_sel, download_list, get_url)

In [ ]:
ds

In [ ]:
variables = list(ds.keys())
var_drop = m.get_dropdown(variables, "Variable: ")
var_drop

In [ ]:
ds_data = ds[var_drop.value].squeeze()

In [ ]:
f = plt.figure(figsize=(10,8))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()
gl= ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True)
gl.right_labels=False
gl.top_labels=False
ax.add_feature(cfeature.LAND, zorder=1,edgecolor='k')
ax.set_extent([12,30, 43, 48],crs=ccrs.PlateCarree())

f.suptitle(ds_data.long_name, fontsize=28)
im = ax.pcolor(ds_data['longitude'].data,ds_data['latitude'].data, ds_data, cmap='plasma')
cbar= f.colorbar(im,ax=ax)
cbar.set_label(ds_data.units, rotation =270, fontsize=23,labelpad=15)

In [ ]:
hv_dataset = hv.Dataset(ds_data)
hv_dataset.data

In [ ]:
ds.hvplot.quadmesh(
    'longitude', 'latitude', var_drop.value, projection=ccrs.Orthographic(7, 30),
    global_extent=True, frame_height=540, cmap='viridis',
    coastline=True
)

In [ ]:
ds.hvplot.quadmesh(
    'longitude', 'latitude', var_drop.value, crs=ccrs.PlateCarree(), projection=ccrs.PlateCarree(),
    global_extent=True, frame_height=400, cmap='viridis',
    coastline=True
)

----

# 4 - MARINE - Analysis/Forecast Sea Bottom Temperature

In [ ]:
dataset_id = m.get_dropdown(ds_list, "List of datasets: ")
dataset_id

Select 'EO:MO:DAT:NWSHELF_ANALYSISFORECAST_PHY_LR_004_001:cmems_mod_nws_phy-bottomt_anfc_7km-2D_P1D-m' from the Dropdown list:

In [ ]:
data_df = m.get_data_from_name_request(2000, dataset_id.value)
dataset_list = data_df["datasetId"]

In [ ]:
description = m.get_description(data_df, dataset_id)

In [ ]:
image = m.display_image(data_df, dataset_id, 500, "jpg", 600 )
image

In [ ]:
%load_ext autoreload
%autoreload 2
import WEkEO_methods as m

In [ ]:
metadata = m.get_metadata(dataset_id, headers)

In [ ]:
variable_list = list(metadata['parameters']['multiStringSelects'][0]['details']['groupedValueLabels'][0]['valuesLabels'].keys())
download_list = ["Download NETCDF"]

In [ ]:
start_date_sel = m.get_date_picker("Select start date: ")
end_date_sel = m.get_date_picker("Select end date: ")
download_sel = m.select_buttons(download_list, "Data download: ", "Download NETCDF")

In [ ]:
end = metadata['parameters']['dateRangeSelects'][0]['details']['end']
start = metadata['parameters']['dateRangeSelects'][0]['details']['start']
print("The start date is "+start+". The end date is "+end)

In [ ]:
cams_map, dc = m.draw_map(50, -10, 4)

cams_map

In [ ]:
coords = dc.last_draw['geometry']['coordinates'][0]
W = coords[1][0]
E = coords[3][0]
N = coords[1][1]
S = coords[3][1]

In [ ]:
title= widgets.HTML('<h2 style="text-align:center;">CAMS European air quality forecasts </h2><hr><h3 style="padding: 10px;">Input variables - Multiple selection</h3>')
title2 = widgets.HTML('<details><h3>'+str(description[0])+'</div><h3></details>')
variables = widgets.VBox([title, start_date_sel, end_date_sel, download_sel])
image_box = widgets.VBox([image])


ui = widgets.AppLayout(header=title2,
          left_sidebar=variables,
          right_sidebar=image_box,
          layout=widgets.Layout(border='solid'))

# compute_button.on_click(compute_button_f)
container = widgets.Box([ui])
display(container)

In [ ]:
job = m.api_query_sea_floor_temperature(dataset_id, start_date_sel, end_date_sel, W, S, E, N, token)

In [ ]:
job

In [ ]:
jobId = job['jobId']
print("The job ID is: "+jobId)

In [ ]:
get_url= m.request_data(jobId, token)

In [ ]:
order = m.data_order(jobId, get_url, token)
order

In [ ]:
headers = {
    'Accept': 'application/x-netcdf'}
response_order = requests.get("https://wekeo-broker-k8s.apps.mercator.dpi.wekeo.eu/databroker/dataorder/download/"+order, headers=headers)

In [ ]:
response_order

In [ ]:
file = open("bottomT.nc", "wb")
file.write(response_order.content)
file.close()

In [ ]:
ds = xr.open_dataset('bottomT.nc')
ds

In [ ]:
variables = list(ds.keys())
var_drop = m.get_dropdown(variables, "Variable: ")
var_drop

In [ ]:
dates = ds.time.values.astype('datetime64[s]').tolist()
date_drop = m.get_dropdown(dates, "Variable: ")
date_drop

In [ ]:
ds_data = ds[var_drop.value].sel(time=date_drop.value).squeeze()

In [ ]:
hv_dataset = hv.Dataset(ds_data)
hv_dataset.data

In [ ]:
map_height=400
map_width=800

hv_tiles_osm = hv.element.tiles.OSM()

hv.opts.defaults(
  hv.opts.Image(alpha=0.7,
                height=map_height, width=map_width, 
                colorbar=True, 
                tools=['hover'], active_tools=['wheel_zoom'],
                title=ds_data.long_name+" ["+ds_data.units+"]"),
  hv.opts.Tiles(active_tools=['wheel_zoom'], height=map_height, width=map_width)
)

hv_image_basic = hv.Image(hv_dataset)
hv_image_basic = hv_image_basic.options(cmap='inferno')

ds = ds_data.rio.write_crs('EPSG:4326')
ds_mercator = ds.rio.reproject('EPSG:3857')

hv_dataset = hv.Dataset(ds_mercator)
hv_image_basic = hv.Image(hv_dataset)
hv_image_basic = hv_image_basic.options(cmap='inferno')

hv.element.tiles.OSM() * hv_image_basic

---

# 5 - LAND - Corine (can't download data)

In [ ]:
name = m.text_widget("Insert data name: ")
name #corine

In [ ]:
data_df = m.get_data_from_name_request(2000, name.value)
dataset_list = data_df["datasetId"]
dataset_id = m.get_dropdown(dataset_list, "List of datasets: ")
dataset_id

In [ ]:
description = m.get_description(data_df, dataset_id)

In [ ]:
image = m.display_image(data_df, dataset_id, 500, "jpg", 600 )
image

In [ ]:
metadata = m.get_metadata(dataset_id, headers)

In [ ]:
formats_list = list(metadata['parameters']['stringChoices'][1]['details']['valuesLabels'].keys())
dataset_list = list(metadata['parameters']['stringChoices'][0]['details']['valuesLabels'].keys())

In [ ]:
formats = m.select_buttons(formats_list, "Choose format: ", formats_list[1]) 
dataset_name = m.get_dropdown(dataset_list, "Choose dataset: ")

In [ ]:
title= widgets.HTML('<h2 style="text-align:center;">Corine Land Cover</h2><hr><h3 style="padding: 10px;">Input variables - Multiple selection</h3>')
title2 = widgets.HTML('<details><h3>'+str(description[0])+'</div><h3></details>')
variables = widgets.VBox([title, dataset_name, formats])
image_box = widgets.VBox([image])

ui = widgets.AppLayout(header=title2,
          left_sidebar=variables,
          right_sidebar=image_box,
          layout=widgets.Layout(border='solid'))

# compute_button.on_click(compute_button_f)
container = widgets.Box([ui])
display(container)

In [ ]:
%load_ext autoreload
%autoreload 2
import WEkEO_methods as m

In [ ]:
job = m.api_query_corine(dataset_id, dataset_name, formats, token)

In [ ]:
jobId = job['jobId']
print("The job ID is: "+jobId)

In [ ]:
get_url = m.request_data(jobId, token)

In [ ]:
get_url

In [ ]:
order = m.data_order(jobId, get_url, token)
order

In [ ]:
# headers = {
#     'Content-Type': 'application/json',
#     'Accept': 'application/json',
#     'authorization': 'Basic '+str(token)}
# dataset = requests.get("https://wekeo-broker-k8s.apps.mercator.dpi.wekeo.eu/databroker/dataorder?status=completed", headers=headers)
# dataset

In [ ]:
headers = {
    'Accept': 'application/zip'}
response_order = requests.get("https://wekeo-broker-k8s.apps.mercator.dpi.wekeo.eu/databroker/dataorder/download/"+order, headers=headers)


In [ ]:
import pathlib
import zipfile
import io 
downloaded_file = pathlib.Path.cwd() / "output.zip"

zipfile.ZipFile(downloaded_file.write_bytes(response_order.content),mode='w')

In [ ]:
import zipfile, io, os

with open('test.zip', 'wb') as out_file:
    zipfile.ZipFile(io.BytesIO(response_order.content), mode='w')

In [ ]:
zipfile.ZipFile(io.BytesIO(response_order.content), mode='w')

In [ ]:
zip_filename = "test.zip"
with open(zip_filename, 'wb') as zfile:
    zfile.write(response_order.content)

EEA credentials not available

----

## Datasets exploration

In [ ]:
dataset = requests.get("https://wekeo-broker-k8s.apps.mercator.dpi.wekeo.eu/databroker/datasets?size=2000")
dataset_text = dataset.text
data = json.loads(dataset_text)

Dataframe containing all the datasets available on WEkEO, with an abstract, a preview image and a title:

In [ ]:
# from JSON to Pandas DataFrame
data_df = pd.json_normalize(data['content'])
data_df

Select the dataset id with a dropdown:

In [ ]:
dataset_list = data_df['datasetId']

datasetId_drop = widgets.Dropdown(
    options=list(dataset_list),
    description='Dataset ID:',
    disabled=False)

datasetId_drop

In [ ]:
datasetId_drop.value

Read abstract corresponding to the dataset ID selected:

In [ ]:
abstract = data_df.loc[data_df['datasetId'] == str(datasetId_drop.value)]

In [ ]:
print(list(abstract["abstract"]))

Plot a preview image of the selected dataset:

In [ ]:
img_url = list(abstract["previewImage"])[0]
im = Image.open(requests.get(img_url, stream=True).raw)
plt.imshow(im)

Request dataset metadata:

In [ ]:
dataset = requests.get("https://wekeo-broker.apps.mercator.dpi.wekeo.eu/databroker/querymetadata/"+str(datasetId_drop.value), headers=headers)
dataset_text = dataset.text
dataset = json.loads(dataset_text)

In [ ]:
dataset

Select dataset category:

In [ ]:
category = dataset['parameters']['multiStringSelects'][0]['details']['groupedValueLabels']
category_list = []
for item in category:
     category_list.append(item["label"])

category_drop = widgets.Dropdown(
    options=category_list,
    description='Category:',
    disabled=False)

category_drop

In [ ]:
category_list
category_n = list(range(len(category_list)))

In [ ]:
category_n

In [ ]:
category_dict = dict(zip(category_list, category_n))
category_dict

In [ ]:
parameter = category[category_dict[str(category_drop.value)]]
parameter

In [ ]:
params_list = list(parameter['valuesLabels'].keys())

In [ ]:
params_drop = widgets.Dropdown(
    options=params_list,
    description='Category:',
    disabled=False)

params_drop

In [ ]:
metadata['parameters']['multiStringSelects'][1]

In [ ]:
metadata

# TEST

In [ ]:
if query['stringChoiceValues'][0]['value'] == 'netcdf':
    fl = matches.results[0]['url']
    print('Reading NetCDF file into memory')
    # load into memory 
    with urlopen(fl) as f:
        ds = xr.open_dataset(f.read()).mean(dim='time')
elif query['stringChoiceValues'][0]['value'] == 'zip':
    print('Downloading zip file')
    matches.download()

In [ ]:
import glob, os

zip_list = []
os.chdir("./")
for file in glob.glob("*.zip"):
    print(file)
    zip_list.append(file)

In [ ]:
zip_list

In [ ]:
import ipywidgets as widgets

zip_drop = widgets.Dropdown(
    options=zip_list,
    description='Zip folder:',
    disabled=False,
)
zip_drop

In [ ]:
import zipfile
with zipfile.ZipFile(str(zip_drop.value), 'r') as zip_ref:
    zip_ref.extractall('./')

In [ ]:
nc_list = []
os.chdir("./")
for file in glob.glob("*.nc"):
    print(file)
    nc_list.append(file)

In [ ]:
nc_list

In [ ]:
nc_drop = widgets.Dropdown(
    options=nc_list,
    description='NetCDF folder:',
    disabled=False,
)
nc_drop

In [ ]:
ds = xr.open_dataset(nc_drop.value).mean(dim='time')

In [ ]:
hv_dataset = hv.Dataset(ds)
hv_dataset.data

In [ ]:
ds_data = ds['crwc'].squeeze()

In [ ]:
map_height=400
map_width=800

hv_tiles_osm = hv.element.tiles.OSM()

hv.opts.defaults(
  hv.opts.Image(alpha=0.7,
                height=map_height, width=map_width, 
                colorbar=True, 
                tools=['hover'], active_tools=['wheel_zoom']),
  hv.opts.Tiles(active_tools=['wheel_zoom'], height=map_height, width=map_width)
)

hv_image_basic = hv.Image(hv_dataset)
hv_image_basic = hv_image_basic.options(cmap='inferno')
hv_image_basic

In [ ]:
ds = ds.rio.write_crs('EPSG:4326')
ds_mercator = ds.rio.reproject('EPSG:3857')
hv_dataset = hv.Dataset(ds_mercator)
hv_image_basic = hv.Image(hv_dataset)
hv_image_basic = hv_image_basic.options(cmap='inferno')
hv.element.tiles.OSM() * hv_image_basic

In [ ]:
ds_data = ds[var_drop.value].squeeze()
ds_data

In [ ]:
# f = plt.figure(figsize=(10,8))
# ax = plt.axes(projection=ccrs.PlateCarree())
# ax.coastlines()
# gl= ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True)
# gl.right_labels=False
# gl.top_labels=False
# ax.add_feature(cfeature.LAND, zorder=1,edgecolor='k')
# ax.set_extent([-180,180, -90, 90],crs=ccrs.PlateCarree())

# f.suptitle(ds_data.long_name, fontsize=28)
# im = ax.pcolor(ds_data['longitude'].data,ds_data['latitude'].data, ds_data, cmap='plasma')
# cbar= f.colorbar(im,ax=ax)
# cbar.set_label(ds_data.units, rotation =270, fontsize=23,labelpad=15)

In [ ]:
# df = m.get_all_data_request(2000)
# dataset_list = df["datasetId"]